In [118]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm0104",
    security_token="EwVPSL02qXM8XQ2pkUdA13aW",
)

Empresas Programas por empresa y empresas desde account

In [119]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
    ]
]
Empresas_SF.shape

(7955, 7)

In [120]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(7957, 5)

In [121]:
#Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas.shape

(7963, 12)

In [122]:
Programas_pacientes = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_pacientes = Programas_pacientes[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A"
    ]
]


fin empresas y comienzo de rescate de pacientes


In [123]:
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        'Phone'
        
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

(1569270, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.

In [124]:
#buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF.shape

(1279947, 4)

Union de pacientes programas con las empresas

In [125]:
#Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_pacientes,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c",
)
empresas_unidas.shape

(1279947, 12)

In [126]:
empresas_unidas = pd.merge(
    left=empresas_unidas,
    right=Pacientes_SF,
    how="left",
    left_on="Paciente__c",
    right_on="Id Account Paciente",
)
empresas_unidas.shape

(1279947, 21)

In [127]:
empresas_unidas = empresas_unidas.rename(
    columns={
        "Id": "Id Pacientes Programas"       
    }
)
empresas_unidas = empresas_unidas[
    [
        "Id Pacientes Programas",
        "Paciente__c",
        "Programas_por_Empresas__c",
        'Id Account Empresa',
        'Nombre Empresa',
        'Holding',
        'Campaña',
        'Empleador',
        'Poliza',
        'Id Account Paciente',
        'Rut',
        'Nombre',
        'Ocupacion',
        'Cargo',
        'Departamento',
        'Sexo',
        'Telefono',
        'Tipo de Documento',
        'Codigo EMP',
        'Codigo A'


    ]]
empresas_unidas.head()

,Id Pacientes Programas,Paciente__c,Programas_por_Empresas__c,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Id Account Paciente,Rut,Nombre,Ocupacion,Cargo,Departamento,Sexo,Telefono,Tipo de Documento,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,0018c00002amLbpAAE,224488556,Genesis Antonia Gutierrez Abarca,None,None,None,Mujer,967786319,RUT,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,0018c00002amLbqAAE,113376066,Claudia Silva,None,None,None,Mujer,None,RUT,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,0018c00002amLbrAAE,79476161,Maria Lorenza Garay,None,None,None,Mujer,None,RUT,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,0018c00002amLbsAAE,181854227,Estolaza Vidal Jean Patrick Moises,None,None,None,Hombre,953757757,RUT,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,0018c00002amLc2AAE,126899823,Rivas Correa Claudia,None,None,None,Mujer,982058845,RUT,EMP6274,A-0353


Terminado la visualizacion de las empresas y pacientes que se muestran en los reportes


Descarga desde Citas de Servicios

In [128]:
# Citas de Servicio
results = sf.query_all(
    """
    SELECT Id, clinica__c,AccountId,CreatedDate,RUT__c,Work_TypeGroup__c,Status,Interacci_n__c,AppointmentNumber,Tipo_de_la_cita__c from ServiceAppointment
    """
)


Citas_de_servicio = pd.DataFrame(results["records"])
Citas_de_servicio['Empresas'] = np.nan
#Citas_de_servicio['RUT__c'].map(pacientessf_empresas.set_index('IdentificationId__pc_x',verify_integrity=False)['Name']) 
Citas_de_servicio['Empresas Id'] = np.nan
Citas_de_servicio = Citas_de_servicio[
    [
        "AppointmentNumber",        
        "RUT__c",  
        'Empresas',      
        "CreatedDate",
        "Tipo_de_la_cita__c",
        'Empresas Id',        
        "AccountId",
        'Clinica__c'
        
    ]
]
Citas_de_servicio.shape


(8769, 8)

In [129]:
# Citas de Servicio Orientacion
results = sf.query_all(
    """
    SELECT Id, AccountId,CreatedDate,RUT__c,WorkTypeId,Work_TypeGroup__c,Status,Interacci_n__c,Tipo_de_la_cita__c,AppointmentNumber from ServiceAppointment
    """
)
Citas_de_servicio2 = pd.DataFrame(results["records"])
Citas_de_servicio2['Clinica__c'] = 'Orientación Médica'
Citas_de_servicio2['Empresas'] = np.nan
Citas_de_servicio2['Empresas Id'] = np.nan
Citas_de_servicio2 = Citas_de_servicio2[
    [
        "AppointmentNumber",        
        "RUT__c",  
        'Empresas',      
        "CreatedDate",
        "Tipo_de_la_cita__c",
        'Empresas Id',       
        "AccountId",
        'Clinica__c',
        
    ]
]
Citas_de_servicio2.shape


(8769, 8)

In [130]:
#unir dos citas
Gran_Citas_deservicio = pd.concat([Citas_de_servicio, Citas_de_servicio2], axis=0)
Gran_Citas_deservicio = Gran_Citas_deservicio.rename(
    columns={
        "AppointmentNumber": "Codigo Ficha",
        "RUT__c": "Rut",
        "Empresas": "Empleador",
        "CreatedDate": "Fecha",
        "Tipo_de_la_cita__c": "Canal",
        "Empresas Id": "Cod. Empresa",
        "AccountId": "Id Paciente",        
        'Clinica__c':'Clínica'
    }
)

Gran_Citas_deservicio.shape

(17538, 8)

In [131]:
#Clinica_deporte
results = sf.query_all(
    """
   SELECT id,Name,Rut__c,Nombre_Empresa__c,CreatedDate,Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Deporte__c
     """
)
ficha_deporte = pd.DataFrame(results["records"])

ficha_deporte = ficha_deporte[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_deporte['clinica'] = 'Deporte'
ficha_deporte = ficha_deporte.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",        
        'clinica':'Clínica'
    }
)
ficha_deporte.shape


(1041, 8)

In [132]:
# Clinica Nutricion
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Clinica_Nutrici_n__c 
     """
)
ficha_nutricion = pd.DataFrame(results["records"])

ficha_nutricion = ficha_nutricion[
    [
        "Name",
        "RUT__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_nutricion["clinica"] = "Nutrición"

ficha_nutricion = ficha_nutricion.rename(
    columns={
        "Name": "Codigo Ficha",
        "RUT__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)


ficha_nutricion.shape

(2334, 8)

In [133]:
#Clinica AppSex
results = sf.query_all(
    """
   select  Name, Rut__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_ApsSex__c 
     """
)
ficha_Apssex = pd.DataFrame(results["records"])

ficha_Apssex = ficha_Apssex[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_Apssex['clinica'] = 'Asesoría Psicologíca en Salud Sexual'
ficha_Apssex = ficha_Apssex.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)

ficha_Apssex.shape

(5, 8)

In [134]:
# Clinica Psicologia
results = sf.query_all(
    """
   select  Name, Rut__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId,Motivo_de_consulta_manifiesto__c from Ficha_Cl_nica_Asistencia_Emocional__c 
     """
)
ficha_Psicologia = pd.DataFrame(results["records"])

ficha_Psicologia = ficha_Psicologia[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        "Motivo_de_consulta_manifiesto__c",
    ]
]
ficha_Psicologia["clinica"] = "Psicología"
ficha_Psicologia = ficha_Psicologia.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "clinica": "Clínica",
        "Motivo_de_consulta_manifiesto__c": "Motivo de consulta",
    }
)
ficha_Psicologia.shape

(1664, 9)

In [135]:
#Clinica Sueño
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_del_Sue_o__c 
     """
)
ficha_Sueno = pd.DataFrame(results["records"])
ficha_Sueno = ficha_Sueno[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
    ]
]
ficha_Sueno['clinica'] = 'Clínica Del Sueño'
ficha_Sueno = ficha_Sueno.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Sueno.shape

(231, 8)

In [136]:
#Clinica PAciente Cronico
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Seg_Paciente_Cr_nico__c  
     """
)
ficha_Cronico = pd.DataFrame(results["records"])
ficha_Cronico = ficha_Cronico[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
    ]
]
ficha_Cronico['clinica'] = 'Pacientes Crónicos'
ficha_Cronico = ficha_Cronico.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Cronico.shape

(952, 8)

In [137]:
#Clinica Telemedicina
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Telemedicina__c   
     """
)
ficha_Telemedicina = pd.DataFrame(results["records"])
ficha_Telemedicina = ficha_Telemedicina[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c"
    ]
]
ficha_Telemedicina['clinica'] = 'Telemedicina'
ficha_Telemedicina = ficha_Telemedicina.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",        
        'clinica':'Clínica'
    }
)
ficha_Telemedicina.shape

(2765, 8)

In [138]:
#Clinica Veterinaria
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Veterinaria__c    
     """
)
ficha_Veterinaria = pd.DataFrame(results["records"])
ficha_Veterinaria = ficha_Veterinaria[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c"
    ]
]
ficha_Veterinaria['clinica'] = 'Veterinaria'
ficha_Veterinaria = ficha_Veterinaria.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Veterinaria.shape

(23, 8)

In [139]:
#Clinica Orientacion
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Orientaci_n_Medica__c     
     """
)
ficha_Orientacion = pd.DataFrame(results["records"])
ficha_Orientacion = ficha_Orientacion[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_Orientacion['clinica'] = 'Orientación Médica'
ficha_Orientacion = ficha_Orientacion.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Orientacion.shape

(34, 8)

In [140]:
#Clinica Front
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Front__c     
     """
)
ficha_Front = pd.DataFrame(results["records"])

ficha_Front = ficha_Front[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c"
    ]
]
ficha_Front['clinica'] = 'Orientación Médica'
ficha_Front = ficha_Front.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Front.shape

(355, 8)

In [141]:
#Clinica_general
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c,Account__c,   Programa__c   from Ficha_Clinica_General__c 
     """
)
ficha_general = pd.DataFrame(results["records"])

ficha_general = ficha_general[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'Programa__c'
        
    ]
]

ficha_general = ficha_general.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",        
        'Programa__c':'Clínica'
    }
)
ficha_general.shape


(1095, 8)

In [142]:
#Clinica Ingreso deportivo
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Ingreso_Deportivo__c    
     """
)
ficha_Ingreso_dep = pd.DataFrame(results["records"])

ficha_Ingreso_dep = ficha_Ingreso_dep[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_Ingreso_dep['clinica'] = 'Ingreso Deporte Seguro'
ficha_Ingreso_dep = ficha_Ingreso_dep.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Ingreso_dep.shape

(237, 8)

In [143]:
Clinicas = pd.concat([ficha_Front, ficha_Ingreso_dep,ficha_Orientacion,ficha_Veterinaria,ficha_Telemedicina,ficha_Cronico,ficha_Sueno,ficha_Psicologia,ficha_Apssex,ficha_nutricion,ficha_deporte,ficha_general], axis=0)
Clinicas.shape

(10736, 9)

In [144]:
Reortes_general = pd.concat([Clinicas,Gran_Citas_deservicio ], axis=0)
Reortes_general.shape

(28274, 9)

In [145]:
Reortes_general["Dia"] = pd.to_datetime(Reortes_general["Fecha"]).dt.day

Reortes_general["Mes"] = pd.to_datetime(Reortes_general["Fecha"]).dt.month

Reortes_general["semana"] = pd.to_datetime(Reortes_general["Fecha"]).dt.month_name()

Reortes_general["Año"] = pd.to_datetime(Reortes_general["Fecha"]).dt.year


# x = date(Citas_de_servicio['Dia'], Citas_de_servicio['Mes'], Citas_de_servicio['Año'] )
Reortes_general["Fecha_chilena"] = pd.to_datetime(
    Reortes_general["Fecha"], dayfirst=True, yearfirst=False, exact=True
)
Reortes_general["Fecha_chilena2"] = Reortes_general["Fecha_chilena"].dt.strftime(
    "%d-%m-%Y"
)

clinicas_ = {
    "Clínica de Nutrición": "Nutrición",
    "Clínica de Deporte": "Deporte",
    "Clínica de Psicología": "Psicología",
    "Clínica Ingreso Deportivo Seguro": "Deporte",
    "Clínica del Sueño": "Clínica Del Sueño",
    "Clínica Seguimiento Pacientes Crónicos": "Pacientes Crónicos",
    "Orientación Médica": "Orientación Médica",
    "Asesoría Deportiva": "Deporte",
    "Asistencia Emocional": "Psicología",
    "Asistencia Nutricional": "Nutrición",
    "Clínica APSSex": "Asesoría Psicologíca en Salud Sexual",
    "Clínica Del Sueño": "Clínica Del Sueño",
    "Clínica Veterinaria": "Veterinaria",
    "Ingreso Deportivo Seguro": "Deporte",
    "Clínica Mis Primeros Pasos": "Mis Primeros Pasos",
}
Reortes_general.replace({"Clínica": clinicas_}, inplace=True)

Canal = {
    "Phone": "Teléfono",
    "Video": "Videollamada",
    "Entrante - Chat": "Entrante - Chat Online",
}
Reortes_general.replace({"Canal": Canal}, inplace=True)


Reortes_general["Fecha_chilena"] = Reortes_general["Fecha_chilena"].dt.tz_localize(None)
Reortes_general.Rut.fillna("0", inplace=True)



In [146]:
Reportes_con_empresas = pd.merge(
    left=Reortes_general,
    right=empresas_unidas,
    how="left",
    left_on="Id Paciente",
    right_on="Id Account Paciente",
)


Reportes_con_empresas.shape

(31787, 35)

In [147]:
Reportes_con_empresas['Rut_x'].str.upper()
Reportes_con_empresas['Rut_y'].str.upper()

Reportes_con_empresas.Empleador_x.fillna(Reportes_con_empresas['Nombre Empresa'], inplace=True)

Salida de datos

In [148]:
Reportes_con_empresas.to_excel(
    "C:/Users/jmartinez/Documents/reportes_completos.xlsx", index=False
)


Programas_pacientes.to_excel("C:/Users/jmartinez/Documents/Empresas.xlsx", index=False)
reportes = Reportes_con_empresas[
    [
        "Codigo Ficha",
        "Rut_x",
        "Nombre",
        "Sexo",
        "Ocupacion",
        "Cargo",
        "Departamento",
        "Campaña",
        "Holding",
        "Empleador_x",
        "Poliza",
        "Fecha",
        "Fecha_chilena",
        "Fecha_chilena2",
        "Dia",
        "Mes",
        "Año",
        "Canal",
        "Clínica",
        "Motivo de consulta",
        "Nombre Empresa",
        "Codigo EMP",
    ]
]
reportes = reportes.rename(
    columns={
        "Rut_x": "Rut",
        "Empleador_x": "Empleador"
        
    }
)
reportes.to_excel("C:/Users/jmartinez/Documents/reportes.xlsx", index=False)



In [81]:
general_todo = reportes[(reportes["Empleador"] == reportes["Nombre Empresa"])]
general_todo.to_excel("C:/Users/jmartinez/Documents/general.xlsx", index=False)

Banco Estado

In [82]:
Banco_Estado = reportes[
    (reportes["Codigo EMP"] == "EMP5906")
    | (reportes["Codigo EMP"] == "EMP5907")
    | (reportes["Codigo EMP"] == "EMP6490")
    | (reportes["Codigo EMP"] == "EMP6754")
    | (reportes["Codigo EMP"] == "EMP6837")
    | (reportes["Codigo EMP"] == "EMP6838")
]
Banco_Estado.to_excel('F:/Reportes SF/Banco Estado Metlife/Banco_Estado_Base.xlsx', index=False)


ZURICH

In [83]:
Zurich = reportes[
    (reportes["Campaña"] == "Zurich")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Zurich.to_excel("F:/Reportes SF/Zurich/Zurich_Base.xlsx", index=False)

Amsa = reportes[
    (reportes["Campaña"] == "Zurich")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
    & (reportes["Holding"] == "Antofagasta Minerals S.A.")
]
Amsa.to_excel("F:/Reportes SF/Zurich/Amsa.xlsx", index=False)

Bechtel = reportes[
    (reportes["Campaña"] == "Zurich")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
    & (reportes["Holding"] == "Bechtel")
]
Bechtel.to_excel("F:/Reportes SF/Zurich/Bechtel.xlsx", index=False)

Pesco

In [84]:
Pesco = reportes[
    (reportes["Campaña"] == "Pesco")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
    
]
Pesco.to_excel("F:/Reportes SF/Pesco/Pesco_Base.xlsx", index=False)

    


Milan Fajanovic

In [85]:
Milan = reportes[
    (reportes["Campaña"] == "Milan Fajanovic")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
    
]
Milan.to_excel("F:/Reportes SF/Milan/Milan_Fajanovic_Base.xlsx", index=False)

Wom

In [86]:
wom = reportes[
    (reportes["Campaña"] == "MetLife")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
    & (reportes["Empleador"] == "Wom S.A.")
]
wom.to_excel("F:/Reportes SF/Wom/Wom_Base.xlsx", index=False)

In [87]:
Forus = reportes[
    (reportes["Campaña"] == "Forus")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
    
]
Forus.to_excel("F:/Reportes SF/Forus/Forus_Base.xlsx", index=False)

In [88]:
Cruz = reportes[
    (reportes["Campaña"] == "Farmacias Cruz Verde")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
    
]
Cruz.to_excel("F:/Reportes SF/Cruz Verde/Cruz_Base.xlsx", index=False)

METLIFE

In [89]:
Metlife = reportes[
    (reportes["Campaña"] == "MetLife")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Metlife.to_excel("F:/Reportes SF/MetLife/Metlife_Base.xlsx", index=False)

Conosur = reportes[
    (reportes["Codigo EMP"] == "EMP5976")
    | (reportes["Codigo EMP"] == "EMP5977")
    | (reportes["Codigo EMP"] == "EMP5978")
    | (reportes["Codigo EMP"] == "EMP5979")
    | (reportes["Codigo EMP"] == "EMP5980")
    | (reportes["Codigo EMP"] == "EMP5981")
    | (reportes["Codigo EMP"] == "EMP5982")
    | (reportes["Codigo EMP"] == "EMP5983")
    | (reportes["Codigo EMP"] == "EMP5984")
    | (reportes["Codigo EMP"] == "EMP5985")
    | (reportes["Codigo EMP"] == "EMP5985")
    | (reportes["Codigo EMP"] == "EMP7431")
    | (reportes["Codigo EMP"] == "EMP2828")
]
Conosur.to_excel("F:/Reportes SF/MetLife/Conosur_Base.xlsx", index=False)

In [90]:
Burgos = reportes[
    
    (reportes["Codigo EMP"] == "EMP5924")
    | (reportes["Codigo EMP"] == "EMP5925")
    | (reportes["Codigo EMP"] == "EMP5926")
    | (reportes["Codigo EMP"] == "EMP5927")
    | (reportes["Codigo EMP"] == "EMP5928")
    | (reportes["Codigo EMP"] == "EMP5971")
    | (reportes["Codigo EMP"] == "EMP5972")
    | (reportes["Codigo EMP"] == "EMP5973")
    | (reportes["Codigo EMP"] == "EMP5974")
    | (reportes["Codigo EMP"] == "EMP5975")
    | (reportes["Codigo EMP"] == "EMP7259")
    | (reportes["Codigo EMP"] == "EMP6536")
    | (reportes["Codigo EMP"] == "EMP6959")
    | (reportes["Codigo EMP"] == "EMP6388")
     
]

Burgos = Burgos[(Burgos["Empleador"] == Burgos["Nombre Empresa"])]
Burgos.to_excel("F:/Reportes SF/MetLife/Burgos_Base.xlsx", index=False)

In [91]:
Ey = reportes[(reportes["Campaña"] == "EY") & (reportes["Empleador"] == reportes["Nombre Empresa"])]
Ey.to_excel("F:/Reportes SF/EY/EY_Base.xlsx", index=False)

In [92]:
Willis = reportes[
    
    (reportes["Codigo EMP"] == "EMP5924")
    | (reportes["Codigo EMP"] == "EMP5925")
    | (reportes["Codigo EMP"] == "EMP5926")
    | (reportes["Codigo EMP"] == "EMP5927")
    | (reportes["Codigo EMP"] == "EMP5928")
    | (reportes["Codigo EMP"] == "EMP5971")
    | (reportes["Codigo EMP"] == "EMP5972")
    | (reportes["Codigo EMP"] == "EMP5973")
    | (reportes["Codigo EMP"] == "EMP5974")
    | (reportes["Codigo EMP"] == "EMP5975")
    | (reportes["Codigo EMP"] == "EMP7259")
    | (reportes["Codigo EMP"] == "EMP6536")
    | (reportes["Codigo EMP"] == "EMP6959")
    | (reportes["Codigo EMP"] == "EMP6388")
    
    | (reportes["Codigo EMP"] == "EMP5694")
    | (reportes["Codigo EMP"] == "EMP5655")
    | (reportes["Codigo EMP"] == "EMP5681")
    | (reportes["Codigo EMP"] == "EMP5682")
    | (reportes["Codigo EMP"] == "EMP5684")
    | (reportes["Codigo EMP"] == "EMP6274")
    | (reportes["Codigo EMP"] == "EMP5685")
    | (reportes["Codigo EMP"] == "EMP5686")
    | (reportes["Codigo EMP"] == "EMP5687")
    | (reportes["Codigo EMP"] == "EMP5688")
    | (reportes["Codigo EMP"] == "EMP5689")
    | (reportes["Codigo EMP"] == "EMP5690")
    | (reportes["Codigo EMP"] == "EMP5691")

    | (reportes["Codigo EMP"] == "EMP5692")
    | (reportes["Codigo EMP"] == "EMP5693")
    | (reportes["Codigo EMP"] == "EMP5694")
    | (reportes["Codigo EMP"] == "EMP5695")
    | (reportes["Codigo EMP"] == "EMP5696")
    | (reportes["Codigo EMP"] == "EMP5697")
    | (reportes["Codigo EMP"] == "EMP5698")
    | (reportes["Codigo EMP"] == "EMP5639")
    | (reportes["Codigo EMP"] == "EMP5640")
    | (reportes["Codigo EMP"] == "EMP5641")
    | (reportes["Codigo EMP"] == "EMP5642")
    | (reportes["Codigo EMP"] == "EMP5643")
    | (reportes["Codigo EMP"] == "EMP5644")

    | (reportes["Codigo EMP"] == "EMP5645")
    | (reportes["Codigo EMP"] == "EMP5646")
    | (reportes["Codigo EMP"] == "EMP5647")
    | (reportes["Codigo EMP"] == "EMP5648")
    | (reportes["Codigo EMP"] == "EMP5649")
    | (reportes["Codigo EMP"] == "EMP5650")
    | (reportes["Codigo EMP"] == "EMP5651")
    | (reportes["Codigo EMP"] == "EMP5652")
    | (reportes["Codigo EMP"] == "EMP5653")
    | (reportes["Codigo EMP"] == "EMP5654")
    | (reportes["Codigo EMP"] == "EMP5655")
    | (reportes["Codigo EMP"] == "EMP7526")
    | (reportes["Codigo EMP"] == "EMP6760")

    | (reportes["Codigo EMP"] == "EMP7678")
    | (reportes["Codigo EMP"] == "EMP6888")
    | (reportes["Codigo EMP"] == "EMP6444")
    | (reportes["Codigo EMP"] == "EMP6406")
    | (reportes["Codigo EMP"] == "EMP5616")
    | (reportes["Codigo EMP"] == "EMP9007")
    
     
]

Willis = Willis[(Willis["Empleador"] == Willis["Nombre Empresa"])]
Willis.to_excel("F:/Reportes SF/MetLife/Willis_Base.xlsx", index=False)

In [93]:
Mercado_Libre = reportes[
        (reportes["Codigo EMP"] == "EMP5617")
    ]

Mercado_Libre = Mercado_Libre[(Mercado_Libre["Empleador"] == Mercado_Libre["Nombre Empresa"])]
Mercado_Libre.to_excel("F:/Reportes SF/Mercado Libre/Mercado_Libre_Base.xlsx", index=False)


In [94]:
Ahumada = reportes[
        (reportes["Codigo EMP"] == "EMP6475")
    ]

Ahumada = Ahumada[(Ahumada["Empleador"] == Ahumada["Nombre Empresa"])]
Ahumada.to_excel("F:/Reportes SF/Farmacias Ahumada/Ahumada_Base.xlsx", index=False)


In [95]:
Minera_Abra = reportes[
        (reportes["Codigo EMP"] == "EMP7397")
        | (reportes["Codigo EMP"] == "EMP7795")
        |(reportes["Codigo EMP"] == "EMP7796")
    ]

Minera_Abra = Minera_Abra[(Minera_Abra["Empleador"] == Minera_Abra["Nombre Empresa"])]
Minera_Abra.to_excel("F:/Reportes SF/Minera El Abra/Abra_Base.xlsx", index=False)

In [96]:
Agrosuper = reportes[
        (reportes["Holding"] == "Agrosuper")
    ]

Agrosuper = Agrosuper[(Agrosuper["Empleador"] == Agrosuper["Nombre Empresa"])]
Agrosuper.to_excel("F:/Reportes SF/Metlife/Mercer_Agrosuper_Base.xlsx", index=False)

In [97]:
Lipigas = reportes[
        (reportes["Codigo EMP"] == "EMP7310")
    ]

Lipigas = Lipigas[(Lipigas["Empleador"] == Lipigas["Nombre Empresa"])]
Lipigas.to_excel("F:/Reportes SF/Lipigas/Lipigas_Base.xlsx", index=False)

In [100]:
Socofar = reportes[
        (reportes["Codigo EMP"] == "EMP3780")
        | (reportes["Codigo EMP"] == "EMP6749")
    | (reportes["Codigo EMP"] == "EMP6750")
    | (reportes["Codigo EMP"] == "EMP6751")
    | (reportes["Codigo EMP"] == "EMP7405")
    | (reportes["Codigo EMP"] == "EMP7406")
    | (reportes["Codigo EMP"] == "EMP7407")
    
    ]

   


Socofar = Socofar[(Socofar["Empleador"] == Socofar["Nombre Empresa"])]
Socofar.to_excel("F:/Reportes SF/Socofar/Socofar_Base.xlsx", index=False)

In [102]:

Kaufmann = reportes[
    (reportes["Codigo EMP"] == "EMP4171")
    | (reportes["Codigo EMP"] == "EMP7308")
    | (reportes["Codigo EMP"] == "EMP7309")
]


Kaufmann = Kaufmann[(Kaufmann["Empleador"] == Kaufmann["Nombre Empresa"])]
Kaufmann.to_excel("F:/Reportes SF/Kaufmann/Kaufmann_Base.xlsx", index=False)

In [103]:

Tressp = reportes[
    (reportes["Codigo EMP"] == "EMP6467")
    | (reportes["Codigo EMP"] == "EMP6468")
    | (reportes["Codigo EMP"] == "EMP7607")
]


Tressp = Tressp[(Tressp["Empleador"] == Tressp["Nombre Empresa"])]
Tressp.to_excel("F:/Reportes SF/3SP/3sp_Base.xlsx", index=False)


BCI


In [104]:
Bci = reportes[
    (reportes["Campaña"] == "BCI")
]


Bci = Bci[(Bci["Empleador"] == Bci["Nombre Empresa"])]
Bci.to_excel("F:/Reportes SF/BCI/BCI_Base.xlsx", index=False)

In [106]:
Colmena = reportes[
    (reportes["Campaña"] == "Colmena")
]


Colmena = Colmena[(Colmena["Empleador"] == Colmena["Nombre Empresa"])]
Colmena.to_excel("F:/Reportes SF/Colmena/Colmena_Base.xlsx", index=False)

In [112]:

Pedro = reportes[
    (reportes["Campaña"] == "Pedro De Valdivia")
]


Pedro = Pedro[(Pedro["Empleador"] == Pedro["Nombre Empresa"])]
Pedro.to_excel("F:/Reportes SF/Pedro de Valdivia/Pedro_Base.xlsx", index=False)

In [113]:

Vidacel = reportes[
    (reportes["Campaña"] == "Vidacel")
]


Vidacel = Vidacel[(Vidacel["Empleador"] == Vidacel["Nombre Empresa"])]
Vidacel.to_excel("F:/Reportes SF/Vidacel/Vidacel_Base.xlsx", index=False)

VidacelColab = reportes[
    (reportes["Campaña"] == "Vidacel Colaboradores")
]

VidacelColab = VidacelColab[(VidacelColab["Empleador"] == VidacelColab["Nombre Empresa"])]
VidacelColab.to_excel("F:/Reportes SF/Vidacel/VidacelColab_Base.xlsx", index=False)


In [114]:

Wfs = reportes[
    (reportes["Campaña"] == "Holding Wfs Food Services")
]

Wfs = Wfs[(Wfs["Empleador"] == Wfs["Nombre Empresa"])]
Wfs.to_excel("F:/Reportes SF/Wfs/Wfs_Base.xlsx", index=False)


In [115]:

Dermic = reportes[
    (reportes["Campaña"] == "Dermic")
]

Dermic = Dermic[(Dermic["Empleador"] == Dermic["Nombre Empresa"])]
Dermic.to_excel("F:/Reportes SF/Dermic/Dermic_Base.xlsx", index=False)

In [116]:

Instruvalve = reportes[
    (reportes["Campaña"] == "Instruvalve")
]

Instruvalve = Instruvalve[(Instruvalve["Empleador"] == Instruvalve["Nombre Empresa"])]
Instruvalve.to_excel("F:/Reportes SF/Instruvalve/Instruvalve_Base.xlsx", index=False)

In [117]:

Thb = reportes[
    (reportes["Campaña"] == "THB Seguros")
]

Thb = Thb[(Thb["Empleador"] == Thb["Nombre Empresa"])]
Thb.to_excel("F:/Reportes SF/THB Seguros/Thb_Base.xlsx", index=False)

In [98]:
Metlife = reportes[
    (reportes["Campaña"] == "MetLife")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Metlife.to_excel("C:/Users/jmartinez/Documents/Metlife.xlsx", index=False)

Security = reportes[
    (reportes["Campaña"] == "Vida Security")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Security.to_excel("C:/Users/jmartinez/Documents/Vida Security.xlsx", index=False)

#Reportes_con_empresas.Empleador_x.fillna(Reportes_con_empresas['Nombre Empresa'], inplace=True)
reportes_francisca = Reportes_con_empresas
reportes_francisca.Empleador_x.fillna('Nones',inplace=True)
reportes_francisca = reportes_francisca[
    reportes_francisca['Empleador_x'] == 'Nones'
]
archivo = 'SinEmpresas.xlsx'
archivo1 = 'Empresas.xlsx'

fecha = str(datetime.date.today())
archivo_con_fecha = f"{fecha}_{archivo}"
archivo_con_fecha = archivo_con_fecha.replace(":", "_")
archivo_con_fecha = archivo_con_fecha.replace("-", "_")
archivo_con_fecha = archivo_con_fecha.replace(" ", "_")
#archivo_con_fecha = archivo_con_fecha.replace(".", "_")
hasta = Path("E:/Siempre/Usuarios/Francisca Rubio Yáñez/TI/Pacientes Sin Empresas/").joinpath(archivo_con_fecha)
print (hasta)
reportes_francisca.to_excel(
    hasta, index=False
)



E:\Siempre\Usuarios\Francisca Rubio Yáñez\TI\Pacientes Sin Empresas\2023_02_13_SinEmpresas.xlsx


In [99]:
fecha = str(datetime.date.today())
archivo_con_fecha = f"{fecha}_{archivo1}"
archivo_con_fecha = archivo_con_fecha.replace(":", "_")
archivo_con_fecha = archivo_con_fecha.replace("-", "_")
archivo_con_fecha = archivo_con_fecha.replace(" ", "_")
hasta = Path("E:/Siempre/Usuarios/Francisca Rubio Yáñez/TI/Empresas/").joinpath(archivo_con_fecha)
Programas_pacientes.to_excel(
    hasta, index=False
)

hasta = Path("F:/Empresas SF/empresas/").joinpath(archivo_con_fecha)
Programas_pacientes.to_excel(
    hasta, index=False
)